In [1]:
# Important standard packages 
import pandas as pd
import numpy as np

In [88]:
# Importing packages used in data pre-processing
import gensim
from gensim.utils import simple_preprocess, tokenize
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
from nltk.corpus import stopwords
import string 

# Preparing data
1. Loading data and dropping irrelevant columns
2. Removal of NaN values from relevant attributes
3. Create new dataframe consisting of only keywords and country ID

In [3]:
data=pd.read_csv("FAOLEX_ALL.csv")
# print(data.keys())
# print(df.info())
# print(keywords[:5])

# Create simplified version of the dataframe by discarding irrelevant columns 
to_drop = ['Record Id', 
           'Record URL', 
           'Document URL', 
           'Title',
           'Original title',
           'Available website',
           'Abstract']

data.drop(to_drop, inplace=True, axis=1)

In [4]:
# Remove NaN values in the Countries/Territory and Keywords attributes 
data_no_nan = data.dropna(subset=["Country/Territory", "Keywords"])
print(f"This results in {len(data) - len(data_no_nan)} rows being dropped")

This results in 939 rows being dropped


In [5]:
data_kw = data_no_nan[["Country/Territory","Keywords"]]
data_kw['index'] = data_kw.index
keywords = data_kw

/var/folders/sb/klhtpy1n7jd649__g8r7xj300000gn/T/ipykernel_25779/1041940241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_kw['index'] = data_kw.index


# Data pre-processing
1. Create various pre-processing functions
2. Perform the pre-processing on the simplified dataframe created above
3. Stopwords removal
4. Lemmatization (yet to be done)

In [71]:
# Create function to remove punctuation from input text
def remove_punctuation(text):
    # signs = ";'-"
    signs = ";"
    #return "".join([i for i in text if i not in string.punctuation])
    
    return "".join([i for i in text if i not in signs])

'''
def remove_stop_words(text): 
    for word in text: 
        if word in stop_words: 
            text = text.replace(word, '')
    return text
'''


# Create function to do data pre-processing (excluding stop word removal and lemmatization)
def pre_process(text):
    text = remove_punctuation(text)
    # text = remove_stop_words(text)
    text = text.lower()
    processed = list(tokenize(text))
    
    return processed


In [77]:
# Performing pre-processsing 
processed_kw = keywords["Keywords"].map(pre_process)

# Convert to a list
processed_kw_list = processed_kw.values.tolist()

# Remove stop words
stop_words = stopwords.words('english')
for lists in processed_kw_list:
    for kw in lists: 
        if kw in stop_words:
            lists.remove(kw)

# Lemmatization 
# TODO figure out a way to do this 


# LDA model

In [82]:
# Mapping from word IDs to words
# https://medium.com/analytics-vidhya/topic-modeling-using-gensim-lda-in-python-48eaa2344920
id2word = corpora.Dictionary(processed_kw_list)

# Prepare Document-Term matrix
corpus = []
for doc in processed_kw_list:
    corpus.append(id2word.doc2bow(doc))

In [84]:
# Creating the LDA model
# Documentation: https://radimrehurek.com/gensim/models/ldamodel.html
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, 
                                            id2word = id2word, 
                                            num_topics=30, 
                                            random_state=100, 
                                            passes = 10)

In [96]:
print(lda_model.print_topics())

[(24, '0.182*"health" + 0.161*"animal" + 0.154*"governance" + 0.141*"monitoring" + 0.129*"disasters" + 0.045*"production" + 0.040*"feed" + 0.035*"feedstuffs" + 0.033*"tourism" + 0.023*"welfare"'), (14, '0.189*"sustainable" + 0.129*"public" + 0.113*"protection" + 0.097*"management" + 0.070*"use" + 0.067*"environment" + 0.056*"biodiversity" + 0.030*"cultural" + 0.028*"heritage" + 0.027*"community"'), (28, '0.380*"land" + 0.094*"agricultural" + 0.055*"tenure" + 0.054*"development" + 0.041*"areas" + 0.038*"taxes" + 0.035*"urban" + 0.027*"public" + 0.026*"ownership" + 0.025*"registration"'), (23, '0.136*"diseases" + 0.119*"pests" + 0.118*"health" + 0.104*"procedural" + 0.104*"matters" + 0.093*"animal" + 0.060*"procedures" + 0.060*"sanitary" + 0.060*"hygiene" + 0.038*"international"'), (19, '0.268*"water" + 0.055*"management" + 0.044*"supply" + 0.042*"resources" + 0.034*"potable" + 0.031*"irrigation" + 0.029*"waterworks" + 0.028*"human" + 0.026*"surface" + 0.026*"groundwater"'), (18, '0.296*

In [90]:
# Visualizing the topics using pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
visualization = pyLDAvis.gensim_models.prepare(
    lda_model, 
    corpus,
    id2word, 
    mds = "mmds", 
    R = 30)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

# Random texting of various functions

In [29]:
from nltk.corpus import stopwords
test = "This is a test. It contains VERY specific words such as Jonathan, Gustav and Jacob"
test = test.lower().split()
test = [word for word in test if not word in set(stopwords.words('english'))]
test1 = ' '.join(test)
print(test)
print(test1)

['test.', 'contains', 'specific', 'words', 'jonathan,', 'gustav', 'jacob']
test. contains specific words jonathan, gustav jacob
